In [20]:
pip install --upgrade scikit-learn numpy


  Using cached numpy-2.2.5-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np

In [139]:
files = [
    {'year': 22, 'visit_file': 'h239g.csv', 'people_file': 'h243.csv'},
    {'year': '21', 'visit_file': 'h229g.csv', 'people_file': 'h233.csv'},
    {'year': '20', 'visit_file': 'h220g.csv', 'people_file': 'h224.csv'},
    {'year': '19', 'visit_file': 'h213g.csv', 'people_file': 'h216.csv'},
    ]
dfs = []

In [140]:
def load_data(visit_file, people_file, year, dfs):
    rename_map = {
        f'OBTC{year}X': 'total_charge',  # Total charge
        f'OBSF{year}X': 'out_of_pocket',  # Out-of-pocket
        f'OBMR{year}X': 'medicare',  # Medicare
        f'OBMD{year}X': 'medicade',  # Medicaid
        f'OBPV{year}X': 'priv_insurance',  # Private insurance
        f'OBVA{year}X': 'VA_CHAMPVA',  # VA/CHAMPVA
        f'OBTR{year}X': 'tricare',  # Tricare
        f'OBOF{year}X': 'other_federal', # Other federal
        f'OBSL{year}X': 'state_local',  # State/local
        f'OBWC{year}X': 'workers_comp',  # Worker’s comp
        f'OBOT{year}X': 'other_insurance',  # Other insurance
        f'PERWT{year}F': 'person_weight',  # Person weight
        f'INSURC{year}': 'insurance'
    }

    insurance_col = f'INSURC{year}'

    df_visits = pd.read_csv(visit_file)
    df_people = pd.read_csv(people_file)
    df_people = df_people[['DUPERSID', insurance_col]]
    df_initial = df_visits.merge(df_people, on='DUPERSID', how='left')
    df_curr = df_initial.copy(deep=True)
    df_curr.rename(columns=rename_map, inplace=True)
    dfs.append(df_curr)

In [143]:
for file in files:
    load_data(file['visit_file'], file['people_file'], file['year'], dfs)

df = pd.concat(dfs, ignore_index=True)
print(df)
df.to_csv("df_all_years.csv", index=False)

            DUID  PID    DUPERSID           EVNTIDX  EVENTRN FFEEIDX  PANEL  \
0        2460002  101  2460002101  2460002101008301        8      -1     24   
1        2460002  101  2460002101  2460002101009001        9      -1     24   
2        2460006  101  2460006101  2460006101009401        7      -1     24   
3        2460006  101  2460006101  2460006101010501        8      -1     24   
4        2460006  101  2460006101  2460006101010601        8      -1     24   
...          ...  ...         ...               ...      ...     ...    ...   
1007501  2469688  102  2469688102  2469688102000201        1      -1     24   
1007502  2469689  101  2469689101  2469689101000101        1      -1     24   
1007503  2469689  101  2469689101  2469689101000301        2      -1     24   
1007504  2469689  101  2469689101  2469689101000401        2      -1     24   
1007505  2469689  101  2469689101  2469689101000501        2      -1     24   

         MPCELIG  MPCDATA  OBDATEYR  ...  insurance

In [144]:
df_full = pd.read_csv("df_all_years.csv")

/var/folders/_t/mfgk4xxn1hb8w2n3w5x65snw0000gp/T/ipykernel_62072/1144292760.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv("df_all_years.csv")


In [158]:
df_working = df_full.copy(deep=True)

#Replaces lab test code 95 (no test received) with code 2 (no)
df_working['LABTEST_M18'] = df['LABTEST_M18'].replace(95, 2.0)
df_working['SONOGRAM_M18'] = df['SONOGRAM_M18'].replace(95, 2.0)
df_working['XRAYS_M18'] = df['XRAYS_M18'].replace(95, 2.0)
df_working['MAMMOG_M18'] = df['MAMMOG_M18'].replace(95, 2.0)
df_working['MRI_M18'] = df['MRI_M18'].replace(95, 2.0)
df_working['EKG_M18'] = df['EKG_M18'].replace(95, 2.0)
df_working['RCVVAC_M18'] = df['RCVVAC_M18'].replace(95, 2.0)

#df['LABTEST_M18'].value_counts()

In [159]:
MEPS_NA = [-1, -7, -8, -15]
df_working.replace(MEPS_NA, np.nan, inplace=True)

insurance_map = {
    1.: '<65 Any Private',
    2.: '<65 Public Only',
    3.: '<65 Uninsured',
    4.: '65+ Medicare Only',
    5.: '65+ Medicare + Private',
    6.: '65+ Medicare + Other Public',
    7.: '65+ Uninsured',
    8.: '65+ No Medicare but Public/Private'
}
df_working['insurance']
df_working['insurance'] = df_working['insurance'].map(insurance_map)

print(df_working['insurance'])

0          65+ Medicare + Other Public
1          65+ Medicare + Other Public
2                      <65 Public Only
3                      <65 Public Only
4                      <65 Public Only
                      ...             
1007501                            NaN
1007502                            NaN
1007503                            NaN
1007504                            NaN
1007505                            NaN
Name: insurance, Length: 1007506, dtype: object


In [163]:
cost_columns = [
    'total_charge',   # Total charge
    'out_of_pocket',   # Out-of-pocket (family)
    'medicare',   # Medicare
    'medicaid',   # Medicaid
    'priv_insurance',   # Private insurance
    'VA_CHAMPVA',   # VA/CHAMPVA
    'tricare',   # Tricare
    'other_federal',   # Other federal
    'state_local',   # State/local gov
    'workers_comp',   # Worker’s comp
    'other_insurance',   # Other insurance
]

df_working['COST_COVERED'] = df_working['total_charge'] - df_working['out_of_pocket']

print(df_working['COST_COVERED'])

0          248.00
1          338.00
2          130.66
3          719.61
4          205.03
            ...  
1007501    140.00
1007502    132.00
1007503    349.83
1007504    349.83
1007505    218.82
Name: COST_COVERED, Length: 1007506, dtype: float64


In [165]:
df_working.to_csv("df_working.csv", index=False)